# Practical Exam: House sales

RealAgents is a real estate company that focuses on selling houses.

RealAgents sells a variety of types of house in one metropolitan area.

Some houses sell slowly and sometimes require lowering the price in order to find a buyer.

In order to stay competitive, RealAgents would like to optimize the listing prices of the houses it is trying to sell.

They want to do this by predicting the sale price of a house given its characteristics.

If they can predict the sale price in advance, they can decrease the time to sale.


## Data

The dataset contains records of previous houses sold in the area.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton'. </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced" (two shared walls), "Semi-detached" (one shared wall), or "Detached" (no shared walls). </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |


# Task 1

The team at RealAgents knows that the city that a property is located in makes a difference to the sale price. 

Unfortuntately they believe that this isn't always recorded in the data. 

Calculate the number of missing values of the `city`. 

 - You should use the data in the file "house_sales.csv". 

 - Your output should be an object `missing_city`, that contains the number of missing values in this column. 

In [122]:
# Use this cell to write your code for Task 1

In [123]:
import pandas as pd
import numpy as np

In [124]:
df = pd.read_csv('house_sales.csv')
df.head(3)

,house_id,city,sale_price,sale_date,months_listed,bedrooms,house_type,area
0,1217792,Silvertown,55943,2021-09-12,5.4,2,Semi-detached,107.8 sq.m.
1,1900913,Silvertown,384677,2021-01-17,6.3,5,Detached,498.8 sq.m.
2,1174927,Riverford,281707,2021-11-10,6.9,6,Detached,542.5 sq.m.


In [125]:
missing_city = df['city'].value_counts()[-1]
missing_city

73

# Task 2 

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "house_sales.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.


| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
| house_id    | Nominal. </br> Unique identifier for houses. </br>Missing values not possible. |
| city        | Nominal. </br>The city in which the house is located. One of 'Silvertown', 'Riverford', 'Teasdale' and 'Poppleton' </br>Replace missing values with "Unknown". |
| sale_price  | Discrete. </br>The sale price of the house in whole dollars. Values can be any positive number greater than or equal to zero.</br>Remove missing entries. |
| sale_date   | Discrete. </br>The date of the last sale of the house. </br>Replace missing values with 2023-01-01. |
| months_listed  | Continuous. </br>The number of months the house was listed on the market prior to its last sale, rounded to one decimal place. </br>Replace missing values with mean number of months listed, to one decimal place. |
| bedrooms    | Discrete. </br>The number of bedrooms in the house. Any positive values greater than or equal to zero. </br>Replace missing values with the mean number of bedrooms, rounded to the nearest integer. |
| house_type   | Ordinal. </br>One of "Terraced", "Semi-detached", or "Detached". </br>Replace missing values with the most common house type. |
| area      | Continuous. </br>The area of the house in square meters, rounded to one decimal place. </br>Replace missing values with the mean, to one decimal place. |

In [126]:
# Use this cell to write your code for Task 2

In [127]:
def wrangle(filepath):
    df = pd.read_csv(filepath)
    # replace city missing values
    df['city'] = df['city'].replace('--','Unknown')
    # datetime column
    df['sale_date'] = pd.to_datetime(df['sale_date'])
    # replace missing values of month_listed by mean
    df['months_listed'].fillna(df['months_listed'].mean(),inplace=True)
    df['months_listed'] = df['months_listed'].round(1)
    # fix house_type values
    types = {'Det.':'Detached','Semi':'Semi-detached','Terr.':'Terraced'}
    df['house_type'].replace(types,inplace=True)
    # fix area
    df['area'] = (df['area']
                  .str.split(expand=True)[0]
                  .astype(float).round(1))
    return df

In [128]:
clean_data = wrangle('house_sales.csv')
df = clean_data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   house_id       1500 non-null   int64         
 1   city           1500 non-null   object        
 2   sale_price     1500 non-null   int64         
 3   sale_date      1500 non-null   datetime64[ns]
 4   months_listed  1500 non-null   float64       
 5   bedrooms       1500 non-null   int64         
 6   house_type     1500 non-null   object        
 7   area           1500 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 93.9+ KB


# Task 3 

The team at RealAgents have told you that they have always believed that the number of bedrooms is the biggest driver of house price. 

Producing a table showing the difference in the average sale price by number of bedrooms along with the variance to investigate this question for the team.

 - You should start with the data in the file 'house_sales.csv'.

 - Your output should be a data frame named `price_by_rooms`. 

 - It should include the three columns `bedrooms`, `avg_price`, `var_price`. 

 - Your answers should be rounded to 1 decimal place.   

In [129]:
# Use this cell to write your code for Task 3

In [130]:
price_by_rooms = (df.groupby('bedrooms')['sale_price']
                  .aggregate(['mean','var'])
                  .reset_index().round(1))
price_by_rooms.rename(columns={'mean':'avg_price','var':'var_price'},inplace=True)
price_by_rooms

,bedrooms,avg_price,var_price
0,2,67076.4,5.652896e+08
1,3,154665.1,2.378289e+09
2,4,234704.6,1.725211e+09
3,5,301515.9,2.484328e+09
4,6,375741.3,3.924432e+09


# Task 4

Fit a baseline model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [131]:
# Use this cell to write your code for Task 4

In [132]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error

In [133]:
train_data = pd.read_csv("train.csv")
val_data = pd.read_csv("validation.csv")

In [134]:
val_data.head()

,house_id,city,sale_date,months_listed,bedrooms,house_type,area
0,1331375,Teasdale,2022-05-17,7.7,3,Terraced,209.7
1,1630115,Teasdale,2020-06-30,6.5,4,Detached,390.6
2,1645745,Silvertown,2020-09-02,7.4,6,Detached,556.8
3,1336775,Silvertown,2021-10-03,8.8,3,Semi-detached,208.3
4,1888274,Silvertown,2022-06-13,5.7,4,Detached,389.2


In [135]:
X = train_data[['sale_price','area']].drop(columns='sale_price')
y = train_data['sale_price']
X_val = val_data[['bedrooms','area']].drop(columns='bedrooms')
base_model = LinearRegression()
base_model.fit(X,y)
base_pred = base_model.predict(X_val)

In [136]:
val_data['price'] = base_pred.round(1)
base_result = val_data[['house_id','price']]
base_result.head()

,house_id,price
0,1331375,119126.3
1,1630115,257153.1
2,1645745,383963.8
3,1336775,118058.1
4,1888274,256084.9


# Task 5

Fit a comparison model to predict the sale price of a house.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “validation.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `house_id` and `price`. The price column must be your predicted values.

In [137]:
# Use this cell to write your code for Task 5

In [138]:
cities = {'Silvertown':1,
          'Teasdale':2,
         'Poppleton':3,
         'Riverford':4,
         'Unknown':5}
types = {'Detached':1,
       'Semi-detached':2,
       'Terraced':3}

In [139]:
train_data['city'].replace(cities,inplace=True)
train_data['house_type'].replace(types,inplace=True)
val_data['city'].replace(cities,inplace=True)
val_data['house_type'].replace(types,inplace=True)

In [140]:
val_data.head()

,house_id,city,sale_date,months_listed,bedrooms,house_type,area,price
0,1331375,2,2022-05-17,7.7,3,3,209.7,119126.3
1,1630115,2,2020-06-30,6.5,4,1,390.6,257153.1
2,1645745,1,2020-09-02,7.4,6,1,556.8,383963.8
3,1336775,1,2021-10-03,8.8,3,2,208.3,118058.1
4,1888274,1,2022-06-13,5.7,4,1,389.2,256084.9


In [141]:
X_train = train_data.drop(columns=['house_id','sale_date','sale_price'])
y_train = train_data['sale_price']
val = val_data.drop(columns=['house_id','sale_date','price'])
comp_model = DecisionTreeRegressor()
comp_model.fit(X_train,y_train)
comp_pred = comp_model.predict(val)

val_data['price'] = comp_pred.round(1)
compare_result = val_data[['house_id','price']]
compare_result.head()

,house_id,price
0,1331375,82462.0
1,1630115,304991.0
2,1645745,413320.0
3,1336775,105120.0
4,1888274,274324.0
